In [46]:
import json
import csv
import re
import pandas as pd
import os
import pickle
from fuzzywuzzy import fuzz

In [47]:
def load_data_csv(note_name):
    csv_file_path = './xhs/notes_' + note_name + ".csv"
    input_list = []
    with open(csv_file_path, 'r', newline='',  encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            desc = [row['\ufeffnote_id'], row['desc'], row['liked_count']]
            # print('desc = ', desc)
            input_list.append(desc)
    return input_list

In [48]:
note_name = 'LA_chinesefood'
input_list = load_data_csv(note_name)

In [49]:
# Assuming you have a JSON file named 'data.json'
with open(note_name + '.json', 'r') as json_file:
    data = json.load(json_file)

In [50]:
for key in data:
    # print(data[key])
    match_data = []
    for note_id, doc, liked_count in input_list:
        partial_similarity_ratio = fuzz.partial_ratio(key.lower(), doc.lower())

        # if partial_similarity_ratio >= 50:
        match_data.append([note_id, doc, liked_count, partial_similarity_ratio])

    match_data = sorted(match_data, key=lambda x: (x[3]))[::-1]
    match_data = match_data[:3]
    while len(match_data) > 1 and match_data[-1][-1] < 70:
        match_data.pop()
    
    match_data = sorted(match_data, key=lambda x: (x[2]))[::-1]
    # print('key = ', key)
    # print('match_data = ', match_data)
    
    match_data = [no[0] for no in match_data]
    data[key]['note'] = match_data
        

In [51]:
with open(f'./{note_name}.json', "w") as json_file:
    json.dump(data, json_file)